# **Bài toán: IEEE-CIS Fraud Detection**

Phân tích xem một giao dịch có phải là gian lận hay không từ các dữ liệu liên quan đến cuộc giao dịch đó.
Dữ liệu ở đây được chia làm 2 bảng ở 2 file data khác nhau: 

**1. Bảng Transaction:**

    * TransactionDT: Thời điểm giao dịch từ một mốc thời gian tham chiếu (không phải là một mốc thời gian thực tế).
    * TransactionAMT: số tiền thanh toán giao dịch bằng USD.
    * ProductCD: Mã của sản phẩm cho mỗi giao dịch.
    * card1 - card6: thông tin thẻ thanh toán, chẳng hạn như loại thẻ, loại thẻ, ngân hàng phát hành, quốc gia, v.v.
    * addr: địa chỉ. Ở đây có 2 địa chỉ là addr1 (địa chỉ khu vực thanh toán), addr2 (quốc gia thanh toán). Cả hai địa chỉ đều dành cho người mua.
    * dist: khoảng cách giữa (không giới hạn) địa chỉ thanh toán, địa chỉ gửi thư, mã zip, địa chỉ IP, vùng điện thoại, v.v.
    * P_ and (R__) emaildomain: miền email của người mua và người nhận
    * C1-C14: counting, chẳng hạn như có bao nhiêu địa chỉ được tìm thấy có liên quan đến thẻ thanh toán, v.v. Ý nghĩa thực tế được bảo mật.
    * D1-D15: Thời gian giao dịch, khoảng thời gian được tính từ giao dịch gần nhất đến giao dịch đang được thực hiện...
    * M1-M9: Thông tin kết nổi, chẳng hạn như thông tin kết nối giữa tên trên thẻ và địa chỉ, ...
    * Vxxx: Vesta đã thiết kế các tính năng phong phú, bao gồm xếp hạng, đếm và các quan hệ thực thể khác. Tất cả các features của Vesta được bắt nguồn dưới dạng số. Một số trong số đó là số lượng đơn đặt hàng trong một nhóm, một khoảng thời gian hoặc điều kiện, vì vậy giá trị là hữu hạn và có thứ tự (hoặc xếp hạng)

    Các categorical features ở bảng này:
        * ProductCD
        * card1 - card6
        * addr1, addr2
        * P_emaildomain
        * R_emaildomain
        * M1 - M9
Train transactions shape: (590540, 394), identity (144233, 41)

**2. Bảng identity:**

Các biến trong bảng này là thông tin nhận dạng - thông tin kết nối mạng (IP, ISP, Proxy, v.v.) và chữ ký số (UA / browser / os / version, v.v.) được liên kết với các giao dịch.
Chúng được thu thập bởi hệ thống chống gian lận của Vesta và các đối tác bảo mật kỹ thuật số.
(Các tên trường được che dấu và từ điển theo cặp sẽ không được cung cấp để bảo vệ quyền riêng tư và thỏa thuận hợp đồng)
 Các categorical features ở bảng này:
 * DeviceType: loại thiết bị
 * DeviceInfo: thông tin về thiết bị (ví dụ như Android 7.1.2, LG-H840 Build/NRD90U...)
 * id_12 - id_38
 
Test transactions shape: (506691, 393), identity (141907, 41)

Mô tả rõ hơn về data: https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203#583227

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import plotly
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
import cufflinks
import cufflinks as cf
import plotly.figure_factory as ff

import gc
gc.enable()

from IPython.display import HTML

In [ ]:
# Hàm biểu đồ cột dành cho việc thống kê
def render(chart, id="vega-chart"):
    """
    Helper function to plot altair visualizations.
    """
    chart_str = """
    <div id="{id}"></div><script>
    require(["vega-embed"], function(vg_embed) {{
        const spec = {chart};     
        vg_embed("#{id}", spec, {{defaultStyle: true}}).catch(console.warn);
        console.log("anything?");
    }});
    console.log("really...anything?");
    </script>
    """
    return HTML(
        chart_str.format(
            id=id,
            chart=json.dumps(chart) if isinstance(chart, dict) else chart.to_json(indent=None)
        )
    )

In [ ]:
# Đường dẫn file data
train_transaction_data_file = "../input/ieee-fraud-detection/train_transaction.csv"
test_transaction_data_file = "../input/ieee-fraud-detection/test_transaction.csv"
train_identity_data_file = "../input/ieee-fraud-detection/train_identity.csv"
test_identity_data_file = "../input/ieee-fraud-detection/test_identity.csv"
sample_submission_file = "../input/ieee-fraud-detection/sample_submission.csv"

# **Load data**

In [ ]:
# Đọc data
train_transaction_data = pd.read_csv(train_transaction_data_file)
train_identity_data = pd.read_csv(train_identity_data_file)
test_transaction_data = pd.read_csv(test_transaction_data_file)
test_identity_data = pd.read_csv(test_identity_data_file)
sample_submission = pd.read_csv(sample_submission_file)
del train_transaction_data_file, test_transaction_data_file, train_identity_data_file, test_identity_data_file, sample_submission_file

# **Khám phá data**

In [ ]:
# Kích cỡ data
print('train_transaction shape is {}'.format(train_transaction_data.shape))
print('test_transaction shape is {}'.format(test_transaction_data.shape))
print('train_identity shape is {}'.format(train_identity_data.shape))
print('test_identity shape is {}'.format(test_identity_data.shape))

**Vấn đề thứ nhất:** Trong bảng có nhiều missing value (hoặc Nan)

Các cách giải quyết cho vấn đề này có thể fill missing value bằng nhưng giá trị hằng số, bằng trung bình, trung vị. Cách giải quyết này còn phụ thộc vào vào những features trong data tính chất và có ý nghĩa như nào với kết quả đầu ra.

In [ ]:
train_transaction_data.head()

In [ ]:
# Một vài thông tin về train_transaction_data
train_transaction_data.info()

In [ ]:
# Số lượng missing value cho từng cột trong train_transaction_data
missing_values_count = train_transaction_data.isnull().sum()
print (missing_values_count[:10])
total_cells = np.product(train_transaction_data.shape)
total_missing = missing_values_count.sum()
print ("Phần trăm của missing data = ",(total_missing/total_cells) * 100)

In [ ]:
# Số lượng missing value cho từng cột trong train_identity_data
missing_values_count = train_identity_data.isnull().sum()
print (missing_values_count)
total_cells = np.product(train_identity_data.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

In [ ]:
# Một vài thông tin về train_transaction_data
train_identity_data.info()

In [ ]:
# Số lượng missing value cho từng cột trong test_identity_data
missing_values_count = test_transaction_data.isnull().sum()
print (missing_values_count[0:10])
total_cells = np.product(test_transaction_data.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

In [ ]:
# Số lượng missing value cho từng cột trong test_identity_data
missing_values_count = test_identity_data.isnull().sum()
print (missing_values_count[0:10])
total_cells = np.product(test_identity_data.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

In [ ]:
del missing_values_count, total_cells, total_missing

**Vấn đề thứ hai:** TransactionID ở tập transaction và identity có nhiều giá trị không giống nhau

In [ ]:
print(np.sum(train_transaction_data['TransactionID'].index.isin(train_identity_data['TransactionID'].index.unique())))
print(np.sum(test_transaction_data['TransactionID'].index.isin(test_identity_data['TransactionID'].index.unique())))

24.4% TransactionIDs trong tập train_transaction (144233 / 590540) có liên kết với train_identity.

28.0% TransactionIDs trong tập test_transaction (141907 / 506691) có liên kết với test_identity.

Để tận dùng được 2 nguồn dữ liệu từ 2 file thì việc merge data ở 2 bảng là điều không tránh khỏi. Thì việc merge 2 bảng trong pandas cũng giống như việc nối bảng trong SQL. Về mặt ý tưởng thì ta sẽ cố gắng giữ kích cỡ mẫu, ta sẽ nối 2 bảng với nhau theo "TransactionID" còn các cột không có transactionID tương ứng ở bên bảng identity thì để giá trị missing value.

**Vấn đề thứ ba:** Mất cân bằng giữa các class.

Hầu hết các giao dịch là không gian lận. Nếu ta sử dụng khung dữ liệu này làm cơ sở cho các mô hình dự đoán và phân tích, ta có thể gặp rất nhiều lỗi và các thuật toán có thể sẽ bị thừa vì nó sẽ "giả định" rằng hầu hết các giao dịch không phải là gian lận. Thì cách xử lý ở đây là có thể ta sẽ dùng các phương pháp giúp class trở nên cân bằng hơn như oversampling, undersampling.

In [ ]:
# Biểu đồ phân bố class trong dataset
counts = train_transaction_data['isFraud'].value_counts().values
ax = sns.barplot([0,1], counts)
ax.set(title='Phân bố của các classes trong tập dữ liệu', xlabel = 'Các classes', ylabel='Số lượng')

print("Tỉ lệ class 0: class 1 là:",len(train_transaction_data[train_transaction_data['isFraud']==1])/len(train_transaction_data)*100,":",
     len(train_transaction_data[train_transaction_data['isFraud']==0])/len(train_transaction_data)*100)

**TransactionDT**

> TransactionDT features là thời điểm giao dịch từ một mốc thời gian tham chiếu (không phải là một mốc thời gian thực tế).


Train: min = 86400 max = 15811131

Test: min = 18403224 max = 34214345

Sự khác biệt giữa train.min () và test.max () là x = 34214345 - 86400 = 34127945 nhưng chúng ta không biết nó tính bằng giây, phút hay giờ.

Nếu TransactionDT có đơn vị là giây thì khoảng thời gian trong dataset sẽ là x/(3600 * 24 *365) = 1.0821 năm. Khi đó ta có thể suy ra những điều sau:

* Khoảng thời gian của total dataset là 394.9993634259259 ngày
* Khoảng thời gian của train set là  181.99920138888888 ngày
* Khoảng thời gian của test set là  182.99908564814814 ngày
* Khoảng cách giữa train và test 30.00107638888889 ngày


In [ ]:
# Dưới đây là biểu đồ phân phối của TransactionDT lúc chưa chuẩn hóa và được chuẩn hóa.
fig, ax = plt.subplots(1, 2, figsize=(18,4))

time_val = train_transaction_data['TransactionDT'].values

# Phân phối của TransactionDT
sns.distplot(time_val, ax=ax[0], color='r')
ax[0].set_title('Distribution of TransactionDT', fontsize=14)
ax[1].set_xlim([min(time_val), max(time_val)])

# Phân phối chuẩn của TransactionDT
sns.distplot(np.log(time_val), ax=ax[1], color='b')
ax[1].set_title('Distribution of LOG TransactionDT', fontsize=14)
ax[1].set_xlim([min(np.log(time_val)), max(np.log(time_val))])

plt.show()


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18,4))

# Phân phối chuẩn của Transaction với isFraud=1
time_val = train_transaction_data.loc[train_transaction_data['isFraud'] == 1]['TransactionDT'].values

sns.distplot(np.log(time_val), ax=ax[0], color='r')
ax[0].set_title('Distribution of LOG TransactionDT, isFraud=1', fontsize=14)
ax[1].set_xlim([min(np.log(time_val)), max(np.log(time_val))])

# Phân phối chuẩn của Transaction với isFraud=0
time_val = train_transaction_data.loc[train_transaction_data['isFraud'] == 0]['TransactionDT'].values

sns.distplot(np.log(time_val), ax=ax[1], color='b')
ax[1].set_title('Distribution of LOG TransactionDT, isFraud=0', fontsize=14)
ax[1].set_xlim([min(np.log(time_val)), max(np.log(time_val))])


plt.show()

Biểu đồ dưới đây mô tả sự khác nhau giữa 2 phân phối TransactionDT của tập train và test. Một điều nữa là dựa vào suy luận ở bên trên là TransactionDT nếu được tính theo giây thì khoảng thời gian thu thập dữ liệu train và test sẽ cách nhau một khoảng là 30 ngày (tính bằng cách lấy test.min() - train.max(), điều này đã được trình bày và tính ở bên trên). Điều này sẽ ảnh hưởng đến việc chọn kĩ thuật cross-validation trên thời gian thực như nào cho phù hợp.

Điều này được phát hiện tại đây:
https://www.kaggle.com/robikscube/ieee-fraud-detection-first-look-and-eda

In [ ]:
# Phân bố của tập train và test transaction data
train_transaction_data['TransactionDT'].plot(kind='hist',
                                        figsize=(15, 5),
                                        label='train',
                                        bins=50,
                                        title='Train vs Test TransactionDT distribution')
test_transaction_data['TransactionDT'].plot(kind='hist',
                                       label='test',
                                       bins=50)
plt.legend()
plt.show()

Trong datasets này có thêm 1 loại features khác cũng liên quan đến vấn đề thời gian là D features - khoảng thời gian được tính từ giao dịch gần nhất đến giao dịch đang được thực hiện.

Dưới đây sẽ là biểu đồ biểu diễn độ tương quan giữa D features với TransactionDT

In [ ]:
# Biểu đồ độ tương quan của Transaction với D features
d_features = list(train_transaction_data.columns[31:46])

for i in d_features:
    cor = np.corrcoef(train_transaction_data['TransactionDT'], train_transaction_data[i])[0,1]
    train_transaction_data.set_index('TransactionDT')[i].plot(style='.', title=i+" corr= "+str(round(cor,3)), figsize=(15, 3))
    test_transaction_data.set_index('TransactionDT')[i].plot(style='.', title=i+" corr= "+str(round(cor,3)), figsize=(15, 3))
    plt.show()

Vấn đề ở đây là D features có nhiều giá trị Nan.

In [ ]:
# Số lương missing values của mỗi cột trong D features
missing_values_count = train_transaction_data[d_features].isnull().sum()
missing_values_count

In [ ]:
# Tổng số missing value
total_cells = np.product(train_transaction_data[d_features].shape)
total_missing = missing_values_count.sum()
# Phần trăm data bị missing

print("Phần trăm data bị missing của D features" ,(total_missing/total_cells) * 100)

Có 58.15% D features bị missing. Dưới đây là biểu đồ tương quan giữa các giá trị D features không missing với TransactionDT

In [ ]:
# Biểu đồ độ tương quan giữa D features và TransactionDT sau khi được xử lý missing bằng cách fill -1
for i in d_features:
    cor_tr = np.corrcoef(train_transaction_data['TransactionDT'], train_transaction_data[i].fillna(-1))[0,1]
    cor_te = np.corrcoef(test_transaction_data['TransactionDT'], test_transaction_data[i].fillna(-1))[0,1]
    train_transaction_data.set_index('TransactionDT')[i].fillna(-1).plot(style='.', title=i+" corr_tr= "+str(round(cor_tr,3))+" || corr_te= "+str(round(cor_te,3)), figsize=(15, 3))
    test_transaction_data.set_index('TransactionDT')[i].fillna(-1).plot(style='.', title=i+" corr_tr= "+str(round(cor_tr,3))+"  || corr_te= "+str(round(cor_te,3)), figsize=(15, 3))
    plt.show()

Theo như kết quả trên thì độ tương quan của TransactionDT và d features không được cao lắm. Về mặt toán học thì có vẻ D features với TransactionDT gần như độc lập với nhau.

In [ ]:
del d_features, cor
gc.collect()

**TransactionAtm**

In [ ]:
train_transaction_data["TransactionAmt"].isnull().sum()

In [ ]:
# Biểu đồ phân phối của TransactionAtm: số tiền giao dịch được tính bằng USD
fig, ax = plt.subplots(1, 2, figsize=(18,4))

time_val = train_transaction_data['TransactionAmt'].values

sns.distplot(time_val, ax=ax[0], color='r')
ax[0].set_title('Phân phối của TransactionAmt', fontsize=14)
ax[1].set_xlim([min(time_val), max(time_val)])

sns.distplot(np.log(time_val), ax=ax[1], color='b')
ax[1].set_title('Phân phối được chuẩn hóa của TransactionAmt', fontsize=14)
ax[1].set_xlim([min(np.log(time_val)), max(np.log(time_val))])

plt.show()

In [ ]:
# Phân phối đã được chuẩn hóa của TransactionAtm với isFraud=1 và isFraud=0
fig, ax = plt.subplots(1, 2, figsize=(18,4))

time_val = train_transaction_data.loc[train_transaction_data['isFraud'] == 1]['TransactionAmt'].values

sns.distplot(np.log(time_val), ax=ax[0], color='r')
ax[0].set_title('Phân phối chuẩn của TransactionAmt với isFraud=1', fontsize=14)
ax[1].set_xlim([min(np.log(time_val)), max(np.log(time_val))])

time_val = train_transaction_data.loc[train_transaction_data['isFraud'] == 0]['TransactionAmt'].values

sns.distplot(np.log(time_val), ax=ax[1], color='b')
ax[1].set_title('Phân phối chuẩn của TransactionAmt với isFraud=0', fontsize=14)
ax[1].set_xlim([min(np.log(time_val)), max(np.log(time_val))])


plt.show()

In [ ]:
del time_val

Trong một giao dịch, thì số tiền thanh toán rất quan trọng

**M features: M1 .. M9**

In [ ]:
m_features = list(train_transaction_data.columns[46:55])
train_transaction_data[m_features].head()

In [ ]:
del m_features
gc.collect()

# Unique Values

**D Features**

In [ ]:
# Số lượng giá trị mỗi cột D feautures trong tập TRAIN
plt.figure(figsize=(10, 7))
d_features = list(train_transaction_data.columns[31:46])
uniques = [len(train_transaction_data[col].unique()) for col in d_features]
sns.set(font_scale=1.2)
ax = sns.barplot(d_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng giá trị mỗi cột D feautures trong tập TRAIN')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

In [ ]:
# Số lượng giá trị mỗi cột D feautures trong tập TEST
plt.figure(figsize=(10, 7))
d_features = list(test_transaction_data.columns[30:45])
uniques = [len(test_transaction_data[col].unique()) for col in d_features]
sns.set(font_scale=1.2)
ax = sns.barplot(d_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng giá trị mỗi cột D feautures trong tập TEST')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center")

Dưới đây là một số gải thuyết về D1 D2 D3:

https://www.kaggle.com/akasyanama13/eda-what-s-behind-d-features

**C features**

In [ ]:
# Số lượng giá trị mỗi cột C feautures trong tập TRAIN
plt.figure(figsize=(10, 7))
c_features = list(train_transaction_data.columns[17:31])
uniques = [len(train_transaction_data[col].unique()) for col in c_features]
sns.set(font_scale=1.2)
ax = sns.barplot(c_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng giá trị mỗi cột C feautures trong tập TRAIN')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center")

In [ ]:
# Hệ số tương quan giữa từng cột trong C features với cột "isFraud"
for i in c_features:
    print(train_transaction_data['isFraud'].corr(train_transaction_data[i].fillna(-1)))

In [ ]:
# Print missing values trong C features
sum = 0
for i in c_features:
    sum += train_transaction_data[i].isnull().sum()

print("Missing values trong các cột C features: ",sum)

Ở đây ta chưa có thông tin gì nhiều về C features, cũng như ý nghĩa thật sự về nó cũng được bảo mật và không được tiết lộ. Nên ta sẽ dựa vào độ tương quan của từng cột C features với label để đánh giá xem nó có thật sự mang lại nhiều thông tin không. Kết quả trên cho thấy độ tương quan luôn gần bằng 0, về mặt toán học thì chúng gần như độc lập với nhau, nhưng ta chưa cơ sở cụ thể để loại bỏ chúng.

In [ ]:
# Số lượng giá trị mỗi cột C feautures trong tập TEST
plt.figure(figsize=(10, 7))
c_features = list(test_transaction_data.columns[16:30])
uniques = [len(test_transaction_data[col].unique()) for col in c_features]
sns.set(font_scale=1.2)
ax = sns.barplot(c_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng giá trị mỗi cột C feautures trong tập TEST')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

**V features**

In [ ]:
# Số lượng giá trị mỗi cột V feautures (V1-V66)
plt.figure(figsize=(35, 8))
v_features = list(train_transaction_data.columns[55:121])
uniques = [len(train_transaction_data[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng mỗi cột trong V features')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center")

In [ ]:
# Số lượng giá trị mỗi cột V feautures (V67-V116)
plt.figure(figsize=(35, 8))
v_features = list(train_transaction_data.columns[121:171])
uniques = [len(train_transaction_data[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng mỗi cột trong V features')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center")

In [ ]:
# Số lượng giá trị mỗi cột V feautures (V117-V166)
plt.figure(figsize=(35, 8))
v_features = list(train_transaction_data.columns[171:221])
uniques = [len(train_transaction_data[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng mỗi cột trong V features')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center")

In [ ]:
# Số lượng mỗi cột trong V features (167-216)
plt.figure(figsize=(35, 8))
v_features = list(train_transaction_data.columns[221:271])
uniques = [len(train_transaction_data[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng mỗi cột trong V features')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

In [ ]:
# Số lượng mỗi cột trong V features (217-266)
plt.figure(figsize=(35, 8))
v_features = list(train_transaction_data.columns[271:321])
uniques = [len(train_transaction_data[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng mỗi cột trong V features')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center")

In [ ]:
# Số lượng mỗi cột trong V features (V267-V336)
plt.figure(figsize=(38, 8))
v_features = list(train_transaction_data.columns[321:391])
uniques = [len(train_transaction_data[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng mỗi cột trong V features')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center")

Vesta đã thiết kế các tính năng phong phú, bao gồm xếp hạng, đếm và các quan hệ thực thể khác. Ý nghĩa của nó cũng chưa được tiết lộ cũng như được khám phá, nhưng ở đây người ta có có nói rằng nó thể hiện các quan hệ thực thể nào đó tức là có thể thông tin ở V features mang lại nó thể hiện mối liên hệ đôi tượng nào đó. Ngoài ra V features có quá nhiều cột (339 cột), có thể ở đây ta có thể dùng thuật toán PCA để lọc các cột thông tin quan trọng.

**id_code**

In [ ]:
train_identity_data.head(2)

In [ ]:
# Một vài thông tin về cột id_31
train_identity_data['id_31'][80:100]

In [ ]:
train_identity_data['id_31'].value_counts()

In [ ]:
# Các loại thiết bị được sử dụng
train_identity_data['DeviceType'].value_counts()

Các cột id feature lưu giữ các thông tin kết nối mạng (IP, ISP, Proxy, v.v.) và chữ ký số (UA / browser / os / version, v.v.). Ở đây có cột id_31 có các thông tin đến hãng sản xuất, browser, cũng như là phiên bản phần mềm, điện thoại được ghi.

Nếu để ý kĩ thì các cột "DeviceType" sẽ lưu thông tin loại thiết bị (chỉ có 2 loại thiết bị là mobile và desktop), còn "DeviceInfo" sẽ lưu thông tin về loại thiết bị được dụng (thuộc hãng nào, thiết bị có id, mã sản phẩm là gì ...)

Vì vậy ở đây ta có thể chỉnh cột id_31 tối giản thông tin của id_31 như chrome, firefox, safari, edge, IE, samsung, opera và "các loại thiết bị khác". Vì các thông kia đã được lưu trong cột DeviceType, DeviceInfo. Ngoài ra "các loại thiết bị khác" ở đây là các thiết bị có số lượng khá ít mạng lại ít giá trị thống kê nên những thiết bị đó ta có thể quy về cùng một loại có khả năng làm giảm lượng số lượng phân loại thiết bị cho bài toán.

In [ ]:
# Số lượng mỗi cột trong id_code features của tập train
plt.figure(figsize=(35, 8))
features = list(train_identity_data.columns[1:39])
uniques = [len(train_identity_data[col].unique()) for col in features]
sns.set(font_scale=1.2)
ax = sns.barplot(features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng mỗi cột trong id_code features TRAIN')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center")

In [ ]:
# Số lượng mỗi cột trong ids features của tập TEST
plt.figure(figsize=(35, 8))
features = list(test_identity_data.columns[1:39])
uniques = [len(test_identity_data[col].unique()) for col in features]
sns.set(font_scale=1.2)
ax = sns.barplot(features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Số lượng mỗi cột trong id_code features TEST')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center")

In [ ]:
# Quy hết các thông tin về browser thành các thông tin về browser ở một số lượng nhất định
train_identity_data.loc[train_identity_data['id_31'].str.contains('chrome', na=False), 'id_31'] = 'Chrome'
train_identity_data.loc[train_identity_data['id_31'].str.contains('firefox', na=False), 'id_31'] = 'Firefox'
train_identity_data.loc[train_identity_data['id_31'].str.contains('safari', na=False), 'id_31'] = 'Safari'
train_identity_data.loc[train_identity_data['id_31'].str.contains('edge', na=False), 'id_31'] = 'Edge'
train_identity_data.loc[train_identity_data['id_31'].str.contains('ie', na=False), 'id_31'] = 'IE'
train_identity_data.loc[train_identity_data['id_31'].str.contains('samsung', na=False), 'id_31'] = 'Samsung'
train_identity_data.loc[train_identity_data['id_31'].str.contains('opera', na=False), 'id_31'] = 'Opera'
train_identity_data['id_31'].fillna("NAN", inplace=True)
train_identity_data.loc[train_identity_data.id_31.isin(train_identity_data.id_31.value_counts()[train_identity_data.id_31.value_counts() < 200].index), 'id_31'] = "Others"

In [ ]:
# Thông tin về cột id_31 sau khi thay đổi thông tin ở bước trên
train_identity_data['id_31'].value_counts()

# Categorical Features

* ProductCD
* emaildomain
* card1 - card6
* addr1, addr2
* M1 - M9
* DeviceType
* DeviceInfo
* id_12 - id_38

**ProductCD**

In [ ]:
train_transaction_data['ProductCD'].isnull().sum()

In [ ]:
train_transaction_data['ProductCD']

In [ ]:
#Bản đồ số lượng trong của productCD trong tập TRAIN và TEST
fig, ax = plt.subplots(1, 2, figsize=(20,5))

sns.countplot(x="ProductCD", ax=ax[0], hue = "isFraud", data=train_transaction_data)
ax[0].set_title('ProductCD train', fontsize=14)
sns.countplot(x="ProductCD", ax=ax[1], data=test_transaction_data)
ax[1].set_title('ProductCD test', fontsize=14)
plt.show()

Như số liệu thống kê ở trên, "ProductCD" có 5 giá trị là "W", "H", "C", "S", "R" và trong đó, "W" là giá trị xuất hiện nhiều nhất. Thông tin cũng không bị missing và thống kê rất rõ ràng, nên ta không phải xử lý cột này.

**Device Type & Device Info**

In [ ]:
# Số lượng deviceType
ax = sns.countplot(x="DeviceType", data=train_identity_data)
ax.set_title('DeviceType', fontsize=14)
plt.show()

In [ ]:
# Số lượng các giá trị của Device info
print ("Unique Devices = ",train_identity_data['DeviceInfo'].nunique())
train_identity_data['DeviceInfo'].value_counts()

In [ ]:
# charts = {}
# for i in ['DeviceType', 'DeviceInfo']:
#     feature_count = train_identity_data[i].value_counts(dropna=False)[:40].reset_index().rename(columns={i: 'count', 'index': i})
#     chart = alt.Chart(feature_count).mark_bar().encode(
#                 x=alt.X(f"{i}:N", axis=alt.Axis(title=i)),
#                 y=alt.Y('count:Q', axis=alt.Axis(title='Count')),
#                 tooltip=[i, 'count']
#             ).properties(title=f"Counts of {i}", width=800)
#     charts[i] = chart
    
# render(charts['DeviceType'] & charts['DeviceInfo'])

**Card**

In [ ]:
card_cols = [c for c in train_transaction_data.columns if 'card' in c]
train_transaction_data[card_cols].head()

In [ ]:
# Số lượng mỗi giá trị xuất hiện trong card1 - card6
cards = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6']
for i in cards:
    print ("Unique ",i, " = ",train_transaction_data[i].nunique())

In [ ]:
train_transaction_data['card4'].fillna("NAN", inplace=True)

In [ ]:
train_transaction_data['card6'].fillna("NAN", inplace=True)

In [ ]:
# Biểu đồ số lượng card4 và card6
fig, ax = plt.subplots(1, 4, figsize=(25,5))

sns.countplot(x="card4", ax=ax[0], data=train_transaction_data.loc[train_transaction_data['isFraud'] == 0])
ax[0].set_title('card4 isFraud=0', fontsize=14)
sns.countplot(x="card4", ax=ax[1], data=train_transaction_data.loc[train_transaction_data['isFraud'] == 1])
ax[1].set_title('card4 isFraud=1', fontsize=14)
sns.countplot(x="card6", ax=ax[2], data=train_transaction_data.loc[train_transaction_data['isFraud'] == 0])
ax[2].set_title('card6 isFraud=0', fontsize=14)
sns.countplot(x="card6", ax=ax[3], data=train_transaction_data.loc[train_transaction_data['isFraud'] == 1])
ax[3].set_title('card6 isFraud=1', fontsize=14)
plt.show()

Suy ra: card6 là loại thẻ, card4 là công ty phát hành thẻ tín dụng. Có thể các chỉ số còn lại liên quan đến ngân hàng, quốc gia phát hành... Ta có thể dựa vào 2 trường data xác định xem gian lận thường được thực hiện như nào.

Ở đây có thể thấy visa, master card là nhưng loại thẻ dễ xảy ra gian lận nhất, và thẻ credit (thanh toán) và thẻ debit (vay tiền) có số lượng gian lận xấp xỉ nhau

In [ ]:
# Thống kê các giá trị riêng biệt của card1 - card3
cards = train_transaction_data.iloc[:,5:8].columns

plt.figure(figsize=(18,8*4))
gs = gridspec.GridSpec(8, 4)
for i, cn in enumerate(cards):
    ax = plt.subplot(gs[i])
    sns.histplot(train_transaction_data.loc[train_transaction_data['isFraud'] == 1][cn], bins=50)
    sns.histplot(train_transaction_data.loc[train_transaction_data['isFraud'] == 0][cn], bins=50)
    ax.set_xlabel('')
    ax.set_title('feature: ' + str(cn))
plt.show()

Ở đây cột card1 được phân vào Categorial nhưng card1 có vẻ giống với dữ liệu liên tục, vì có 13553 giá trị riêng biệt.

Hãy xem post này: https://www.kaggle.com/c/ieee-fraud-detection/discussion/100340#latest-578626

**Email Domain**

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(32,10))

# Biểu đổ số lượng từng email domain trong tập train
sns.countplot(y="P_emaildomain", ax=ax[0], data=train_transaction_data)
ax[0].set_title('P_emaildomain', fontsize=14)
# Biểu đổ số lượng từng email domain với isFraud=1 trong tập train
sns.countplot(y="P_emaildomain", ax=ax[1], data=train_transaction_data.loc[train_transaction_data['isFraud'] == 1])
ax[1].set_title('P_emaildomain isFraud = 1', fontsize=14)
# Biểu đổ số lượng từng email domain với isFraud=0 trong tập train
sns.countplot(y="P_emaildomain", ax=ax[2], data=train_transaction_data.loc[train_transaction_data['isFraud'] == 0])
ax[2].set_title('P_emaildomain isFraud = 0', fontsize=14)
plt.show()

Ở đây có thể thấy một số trường email được sử dụng nhiều đó là: gmail.com, yahoo.com, hotmail.com, anonymous. Mô hình có thể tập trung vào những giá trị này để cho ra kết quả chỉnh xác
Có vẻ như những kẻ tội phạm gian lận thích gian lận qua gmail hơn hay tỉ lệ gian lận xảy ra thông qua con đường gmail là nhiều hơn.

Một số dạng email cùng một công ty:

yahoo / ymail / frontier / rocketmail -> Yahoo

hotmail / outlook / live / msn -> Microsoft

icloud / mac / me -> Appe

prodigy / att / sbcglobal-> AT&T

centurylink / embarqmail / q -> Centurylink

aim / aol -> AOL

twc / charter -> Spectrum

Chi tiết ở: https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654

Giống như ý tưởng với trường id_31, việc quy chuẩn về một công ty ta có thể tăng khả năng tìm kiếm gian lận trở nên dễ dàng hơn. Tức là ta sẽ tạo ra một features mới quy hết các mail domain này về cùng 1 dạng như ý tưởng ở trên. Điều khác ở đây là ta tạo thêm features mới chứ không xóa như id_31. Cụ thể ở đây ta sẽ tạo ra 2 features mới: 1 để lưu các domain email, hai là để lưu các hậu tố như .com, .mx, .es, ...

In [ ]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}

us_emails = ['net', 'edu']

for c in ['P_emaildomain', 'R_emaildomain']:
    #Lấy ra các lọc ra các domain email
    train_transaction_data[c + '_bin'] = train_transaction_data[c].map(emails)
    test_transaction_data[c + '_bin'] = test_transaction_data[c].map(emails)
    
    #lấy các hậu tố trong emaildomain
    train_transaction_data[c + '_suffix'] = train_transaction_data[c].map(lambda x: str(x).split('.')[-1])
    test_transaction_data[c + '_suffix'] = test_transaction_data[c].map(lambda x: str(x).split('.')[-1])
    
    train_transaction_data[c + '_suffix'] = train_transaction_data[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test_transaction_data[c + '_suffix'] = test_transaction_data[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

In [ ]:
train_transaction_data['P_emaildomain_bin'].value_counts()

In [ ]:
train_transaction_data['P_emaildomain_suffix'].value_counts()

# Feature engineering and modeling

Thử model vs PCA:

https://www.kaggle.com/viethoang303/iee-modeling-pca

Thử model vs phương pháp class balance:

https://www.kaggle.com/viethoang303/ieee-modeling